In [11]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from typing import Callable

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    graph_builder = StateGraph(State)

    tools = [TavilySearchResults(max_results=2)]

    llm = ChatOpenAI(model_name=model_name)
    llm_with_tools = llm.bind_tools(tools)

    chatbot: Callable[[State], dict] = lambda state: {"messages": [llm_with_tools.invoke(state["messages"])]}
    graph_builder.add_node("chatbot", chatbot)

    tool_node = ToolNode(tools)
    graph_builder.add_node("tools", tool_node)

    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    graph_builder.set_entry_point("chatbot")

    graph = graph_builder.compile(checkpointer=MemorySaver())
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph: StateGraph, user_input: str):
    # ソースコードを記述
    events = graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
# ソースコードを記述
graph = build_graph(MODEL_NAME)

# メインループ
# ソースコードを記述
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(graph, user_input)

こんにちは！
こんにちは！今日はどのようにお手伝いできますか？
1たす2は？
1たす2は3です。
台湾観光について検索結果を教えて

[{"url": "https://www.taipeinavi.com/", "content": "2025-05-06(火)【台湾新名所】日月潭の旅の途中に立ち寄りたい複合スポット「日月町HIZU... 2025-04-29(火)【台湾旅行】人気旅行先「九份」でキュートな台湾土産を買うなら「九份好好 ... 台北 【台湾スタバ】日本では手に入らない！ 5/2 「STARBUCKS × miffy」コラボグッズ販売 【台湾スタバ】日本では手に入らない！ 5/2 「STARBUCKS × miffy」コラボグッズ販売 【日台友好】4/13～10/13期間限定！台湾の人気ブランド「神農生活」が2025年大阪・関西万博に出店 【台湾スタバ】日本では手に入らない！ 5/2 「STARBUCK... スパロクシタン(中山店) 気持ちいい×きれいが同時に叶う！自然派コスメ「ロクシタン(L'OCCITAN... スパロクシタン(統一時代店) 【台湾音楽】4/1、盧廣仲(クラウド・ルー)が新アルバムから『愚... 【台湾スタバ】4/2「GMoG(Global Month of ... スパロクシタン(中山店) 気持ちいい×きれいが同時に叶う！自然派コスメ「ロクシタン(L'OCCITAN... スパロクシタン(統一時代店) 【台湾音楽】4/1、盧廣仲(クラウド・ルー)が新アルバムから『愚人節快樂』をデジタル先行配信！ 【台湾ICカード】台湾のセブンイレブンの端末「i bon」でICカードを予約しよう！ 【台湾ICカード】台湾のコンビニ「セブンイレブン」の端末「i bon」でICカードを予約しよう！ 【2025年】台湾のちまきコレクション③台北福華大飯店The Howard Plaza Hotel Taipei のシェフが教えてくれた「広東粽(ちまき)」の作り方！ 【2025年】台湾のちまきコレクション②台北福華大飯店The Howard Plaza Hotel Taipei のシェフ直伝の台湾北部粽の包み方！ 【2025年】台湾のちまきコレクション①台北福華大飯店The Howard Plaza Hotel Taipei の粽を見てみよう♡ 蘇打綠